In [4]:
!pip install ipywidgets --quiet

import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import tempfile
import os

# Supported languages
languages = {
    'Python': {'ext': '.py', 'command': 'python3 {file}'},
    'C': {'ext': '.c', 'command': 'gcc {file} -o {bin} && {bin}'},
    'C++': {'ext': '.cpp', 'command': 'g++ {file} -o {bin} && {bin}'},
    'Java': {'ext': '.java', 'command': 'javac {file} && java {classname}'},
    'JavaScript (Node.js)': {'ext': '.js', 'command': 'node {file}'},
    'Bash': {'ext': '.sh', 'command': 'bash {file}'}
}

# Widgets
lang_selector = widgets.Dropdown(
    options=list(languages.keys()),
    description='Language:',
    style={'description_width': 'initial'}
)

code_input = widgets.Textarea(
    value='print("Hello World")',
    placeholder='Write your code...',
    layout=widgets.Layout(width='100%', height='200px')
)

run_button = widgets.Button(
    description='▶️ Run',
    button_style='success',
    icon='play'
)

output = widgets.Output()

# Runner
def run_code(_):
    output.clear_output()
    lang = lang_selector.value
    code = code_input.value
    config = languages[lang]

    with tempfile.TemporaryDirectory() as tmpdir:
        ext = config['ext']
        file_path = os.path.join(tmpdir, 'main' + ext)
        with open(file_path, 'w') as f:
            f.write(code)

        try:
            if lang == 'Java':
                classname = 'main'
                cmd = config['command'].format(file=file_path, classname=classname)
            elif lang in ['C', 'C++']:
                bin_path = os.path.join(tmpdir, 'main.out')
                cmd = config['command'].format(file=file_path, bin=bin_path)
            else:
                cmd = config['command'].format(file=file_path)

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, cwd=tmpdir)
            with output:
                if result.stdout:
                    print(result.stdout)
                if result.stderr:
                    print("⚠️ Error:\n" + result.stderr)

        except Exception as e:
            with output:
                print(f"Exception: {e}")

# Bind
run_button.on_click(run_code)

# Display
display(widgets.VBox([lang_selector, code_input, run_button, output]))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
